In [ ]:
import classes.QSys as qsys
import classes.QSim as qsim
import QuantumToolbox as qtbox
import QuantumToolbox.operators as oper
import scipy.sparse as sp
import QuantumToolbox.states as states
from functools import partial
from multiprocessing import Pool, cpu_count
import QuantumToolbox.functions as qFncs
import QuantumToolbox.operators as qOps
import QuantumToolbox.liouvillian as lio
import Plotting.SimplePlots as sPlt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
resonatorDimension = 200
g = 1.79
qfreq = 0
resFreq = 2
irreg = False

In [ ]:
def digitalRabi(obj, stepSize):
    HamJC = 2 * np.pi * (((obj.subSystems[0].frequency/2)*obj.subSystems[0].freeMat()) + obj.couplingHam)
    UnitaryJC = lio.Liouvillian(HamJC, timeStep=(stepSize / 2))
    UnitaryAJC = (UnitaryJC @ UnitaryJC)
    Unitary = UnitaryJC @ obj.sigmaX @ UnitaryAJC @ obj.sigmaX @ UnitaryJC
    return Unitary

In [ ]:
JCSys = qsys.QuantumSystem()

cav = qsys.Cavity(dimension=resonatorDimension, frequency=resFreq)
JCSys.addSubSys(cav)

qub = JCSys.createSubSys(subClass=qsys.Qubit)
qub.frequency = qfreq

In [ ]:
JCSys.coupleBy(qub, cav,'JC', g)

JCSys.initialState = sp.kron(states.basis(cav.dimension, 0), states.basis(2, 1), format='csc')

In [ ]:
qSim = qsim.Simulation(JCSys)
qSim.sweepKey = 'frequency'

cavParity = qOps.parityEXP(cav.freeMat())
p = Pool(processes=cpu_count())
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationList, cavParity),statesDigit)

sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit,irreg)
print(JCSys.Couplings)

In [ ]:
JCSys.reset()
JCSys.addCoupling([qub,cav],[qtbox.operators.sigmax, qtbox.operators.create], g)
JCSys.addCoupling([cav, qub],[qtbox.operators.destroy, qtbox.operators.sigmax], g)

print(JCSys.couplingName)

In [ ]:
qSim.qSys = JCSys
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit,irreg)
print(JCSys.Couplings)

In [ ]:
JCSys.reset(to='JC')
#JCSys.addCoupling([qub,cav],[qtbox.operators.destroy, qtbox.operators.create], g)
#JCSys.addCoupling([cav, qub],[qtbox.operators.destroy, qtbox.operators.create], g)
JCSys.sigmaX = sp.kron(oper.identity(cav.dimension), oper.sigmax())
JCSys.Unitaries = digitalRabi

In [ ]:
qSim.qSys = JCSys
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
print('calculating')
parityDigit = p.map(partial(qFncs.expectationList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit,irreg)
print(JCSys.Couplings)

In [ ]:
JCSys.reset(to=1)

qSim.qSys = JCSys

statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit,irreg)
print(JCSys.Couplings)

In [ ]:
JCSys.reset(to='JC')
qSim.qSys = JCSys
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit,irreg)
print(JCSys.Couplings)

In [ ]:
JCSys.reset(to=2)
print(JCSys.couplingName)
qSim.qSys = JCSys
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit,irreg)
print(JCSys.Couplings)